In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# merge_bms_into_semantic_minio.py — ajoute les points BMS au TTL sémantique (room+weather) puis upload MinIO

import os, sys, argparse, json
from pathlib import Path
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS, XSD

# ==== Namespaces ====
BRICK = Namespace("https://brickschema.org/schema/1.1/Brick#")
BF    = Namespace("https://brickschema.org/schema/BrickFrame#")
QUDT  = Namespace("http://qudt.org/schema/qudt/")
UNIT  = Namespace("http://qudt.org/vocab/unit/")
EX    = Namespace("http://example.org/training#")

# ==== MinIO ====
def s3_client(endpoint, access, secret, secure):
    import boto3
    from botocore.config import Config
    return boto3.client("s3",
        endpoint_url=endpoint, aws_access_key_id=access, aws_secret_access_key=secret,
        use_ssl=bool(secure), verify=bool(secure),
        region_name="us-east-1", config=Config(signature_version="s3v4"))

def ensure_bucket(s3, bucket):
    import botocore
    try: s3.head_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError: s3.create_bucket(Bucket=bucket)

def s3_upload(s3, bucket, p:Path, key:str):
    s3.upload_file(str(p), bucket, key)

def add(g,s,p,o): g.add((s,p,o))
def label(g,s,t): add(g,s,RDFS.label,Literal(t))

def first_or(g, tp, fallback):
    x = list(g.subjects(RDF.type, tp))
    return x[0] if x else fallback

def ensure_node(g, uri, tp, lbl=None):
    add(g, uri, RDF.type, tp)
    if lbl: label(g, uri, lbl)
    return uri

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--month", type=str, default="2025-03")
    # entrée: TTL sémantique déjà fusionné avec météo
    ap.add_argument("--in_sem_ttl", type=str, default="~/DTE/jne_project/semantic/{month}/training_room_semantic.ttl")
    # URI CSV REFINED des mesures BMS
    ap.add_argument("--bms_ref_csv_uri", type=str, default="minio://refined/bms/{month}/bms_refined.csv")
    # sorties locales
    ap.add_argument("--sem_base", type=str, default="~/DTE/jne_project/semantic")
    # MinIO (bucket semantic)
    ap.add_argument("--endpoint", type=str, default=os.environ.get("MINIO_ENDPOINT","http://192.168.0.173:9000"))
    ap.add_argument("--access",   type=str, default=os.environ.get("MINIO_ROOT_USER","minioadmin"))
    ap.add_argument("--secret",   type=str, default=os.environ.get("MINIO_ROOT_PASSWORD","minioadmin"))
    ap.add_argument("--bucket",   type=str, default="semantic")
    ap.add_argument("--prefix",   type=str, default="jne_project/semantic")
    ap.add_argument("--secure",   action="store_true")
    ap.add_argument("--no-upload", action="store_true")
    args,_ = ap.parse_known_args()

    month = args.month
    in_ttl = Path(args.in_sem_ttl.format(month=month)).expanduser().resolve()
    sem_dir  = Path(args.sem_base).expanduser().resolve()/month
    meta_dir = Path(args.sem_base).expanduser().resolve()/"meta"/month
    sem_dir.mkdir(parents=True, exist_ok=True); meta_dir.mkdir(parents=True, exist_ok=True)

    # charger TTL
    g = Graph()
    for pfx,ns in [("brick",BRICK),("bf",BF),("qudt",QUDT),("unit",UNIT),("ex",EX),("rdfs",RDFS)]:
        g.bind(pfx, ns)
    g.parse(str(in_ttl), format="turtle")

    # ressources
    building = first_or(g, BRICK.Building, EX["Building_ENSMR"])
    room     = first_or(g, BRICK.Room, EX["Room_101"])
    hvac_sys  = ensure_node(g, EX["HVAC_System"], BRICK.HVAC_System, "HVAC System")
    light_sys = ensure_node(g, EX["Lighting_System"], BRICK.Lighting_System, "Lighting System")

    # équipements (créés si absents)
    ac = first_or(g, BRICK.Air_Conditioner, EX["AC_Unit_1"])
    add(g, ac, RDF.type, BRICK.Air_Conditioner); label(g, ac, "AC Unit 1")
    add(g, ac, BRICK.hasLocation, room); add(g, hvac_sys, BRICK.hasPart, ac)

    lum = first_or(g, BRICK.Luminaire, EX["LED_Luminaire_1"])
    add(g, lum, RDF.type, BRICK.Luminaire); label(g, lum, "LED Luminaire 1")
    add(g, lum, BRICK.hasLocation, room); add(g, light_sys, BRICK.hasPart, lum)

    bms_uri = args.bms_ref_csv_uri.format(month=month)

    # définition des points BMS: (name, class, unit, csv_col, isPointOf, attach_system)
    points = [
        ("Zone_Temp_Sensor",      BRICK.Temperature_Sensor,       UNIT.DEG_C,   "T_int",         ac,   hvac_sys),
        ("Zone_Temp_Setpoint",    BRICK.Temperature_Setpoint,     UNIT.DEG_C,   "T_set",         ac,   hvac_sys),
        ("HVAC_State",            BRICK.Binary_Sensor,            UNIT.UNITLESS,"hvac_state",    ac,   hvac_sys),
        ("Lighting_State",        BRICK.Binary_Sensor,            UNIT.UNITLESS,"lighting_state", lum,  light_sys),
        ("HVAC_Power_Sensor",     BRICK.Electrical_Power_Sensor,  UNIT.KiloW,   "P_hvac",        ac,   hvac_sys),
        ("Lighting_Power_Sensor", BRICK.Electrical_Power_Sensor,  UNIT.KiloW,   "P_lighting",    lum,  light_sys),
        ("Plug_Power_Sensor",     BRICK.Electrical_Power_Sensor,  UNIT.KiloW,   "P_plug",        room, None),
        ("Total_Power_Sensor",    BRICK.Electrical_Power_Sensor,  UNIT.KiloW,   "P_total",       room, None),
    ]

    for name, klass, unit_uri, col, target, sys_node in points:
        s = EX[name]
        add(g, s, RDF.type, klass); label(g, s, name.replace("_"," "))
        add(g, s, BRICK.hasLocation, room)
        # rattachement
        if target is not None:
            add(g, s, BRICK.isPointOf, target)
        if sys_node is not None:
            add(g, sys_node, BRICK.hasPoint, s)
        # unité + timeseriesId
        add(g, s, QUDT.unit, unit_uri)
        add(g, s, BF.hasTimeseriesId, Literal(f"{bms_uri}::{col}", datatype=XSD.string))

    # sauvegarde
    out_ttl = sem_dir/"training_room_semantic_bms.ttl"
    g.serialize(destination=str(out_ttl), format="turtle")

    manifest = {
        "version":"1.1","month":month,
        "input_semantic_weather": str(in_ttl),
        "bms_csv_refined": bms_uri,
        "output_ttl": str(out_ttl),
        "points": [dict(name=n, col=c) for (n,_,_,c,_,_) in points],
        "ts_id_pattern":"<csv_uri>::<column>"
    }
    man_path = meta_dir/"semantic_manifest_bms.json"
    Path(man_path).write_text(json.dumps(manifest, indent=2), encoding="utf-8")

    # upload
    if not args.no_upload:
        try:
            s3 = s3_client(args.endpoint, args.access, args.secret, args.secure)
            ensure_bucket(s3, args.bucket)
            root = args.prefix.strip("/")
            s3_upload(s3, args.bucket, out_ttl, f"{root}/{month}/training_room_semantic_bms.ttl")
            s3_upload(s3, args.bucket, Path(man_path), f"{root}/meta/{month}/semantic_manifest_bms.json")
            print("minio:", f"s3://{args.bucket}/{root}/{month}/training_room_semantic_bms.ttl")
        except Exception as e:
            print("ERREUR MinIO:", e, file=sys.stderr); sys.exit(3)

    print("local:", out_ttl, "|", man_path)

if __name__ == "__main__":
    main()


minio: s3://semantic/jne_project/semantic/2025-03/training_room_semantic_bms.ttl
local: /home/amina/DTE/jne_project/semantic/2025-03/training_room_semantic_bms.ttl | /home/amina/DTE/jne_project/semantic/meta/2025-03/semantic_manifest_bms.json
